In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [0]:
%pip install -r ../requirements.txt

In [0]:
import pandas as pd
from weather_utilities.coffee_weather import find_region_latlong, pull_weather_data

In [0]:
coffee_data = spark.read.table("dnb_wave_5.sandbox.df_arabica")

In [0]:
%sql
select * from dnb_wave_5.sandbox.coffee_weather_refined

Databricks data profile. Run in Databricks to view.

In [0]:
unique_regions = coffee_data.select("Country of Origin").distinct().collect()
region_list = [row["Country of Origin"] for row in unique_regions]

In [0]:
geocodes = find_region_latlong(region_list)
latitude = [loc.latitude for location_dict in geocodes for country, loc in location_dict.items()]
longitude = [loc.longitude for location_dict in geocodes for country, loc in location_dict.items()]

location_weather_data = pull_weather_data(latitude, longitude)

In [0]:
df = pd.DataFrame(location_weather_data)
df['date'] = df['date'].apply(lambda x: [date.date() for date in x])

spark_df = spark.createDataFrame(df)

In [0]:
extracted_info = [{"country": country, "latitude": loc.latitude, "longitude": loc.longitude} for location_dict in geocodes for country, loc in location_dict.items()]
region_location_map = spark.createDataFrame(extracted_info)

In [0]:
joined = spark_df \
    .join(region_location_map, spark_df.latitude == region_location_map.latitude, how='left') \
    .drop(region_location_map.latitude, region_location_map.longitude)

In [0]:
joined.write.mode("ignore").format("delta").saveAsTable("dnb_wave_5.sandbox.location_weather_raw")